In [17]:
from converter.pgn_data import PGNData
import pandas as pd
import chess.pgn
import pandas as pd
import re
import chess
import chess.engine
from stockfish import Stockfish
engine = chess.engine.SimpleEngine.popen_uci("stockfish-windows-x86-64-modern/stockfish/stockfish-windows-x86-64-modern")
stockfish = Stockfish(path= "stockfish-windows-x86-64-modern/stockfish/stockfish-windows-x86-64-modern")

## Convert PGN to CSV

In [ ]:
# Convert PGN to CSV
# pgn_data = PGNData("fics2020January.pgn")
# pgn_data.export()

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Load the two files created by export() to pandas
game_info_df = pd.read_csv("fics2020January_game_info.csv")
game_moves_df = pd.read_csv("fics2020January_moves.csv")

## Get info db and moves db, primary key game_id

In [3]:
info_cols = ["game_id", "white", "black", "white_elo", "black_elo", "winner"]
info_df = game_info_df[info_cols]
info_df.head()

,game_id,white,black,white_elo,black_elo,winner
0,ace8ccd9-9e96-45d3-b048-258be0c3fb01,Sarabtl,KINGJarthur,1499,1513,KINGJarthur
1,4a1b6302-7aac-4323-9ea5-559185329686,JosephAquilino,jejudoug,1745,1952,JosephAquilino
2,57bc7ff1-8f4d-45ee-8f42-783bb0134881,utigard,Fiddler,1652,1666,utigard
3,64d64692-ec0d-47fd-98ac-b34e9e200fe7,Disfunction,Sandevil,1592,1703,Disfunction
4,bd63e22e-2de4-4640-bd28-7e6604a36e36,KINGJarthur,ShyKnight,1517,1723,ShyKnight


## Adding the move times

In [5]:
moves_df = game_moves_df

def get_times(moves):
    return re.findall("\d+\.\d+", moves)


pgn = open("fics2020January.pgn")

# info_list = []
times_list = []
game_id = 0

#for i in pgn:
for i in pgn:
    lines = str(chess.pgn.read_game(pgn)).split("\n")
    
    for time in get_times(lines[-1]):
        times_list.append(time)



In [6]:
print(len(times_list))
print(len(moves_df))

3640716
3640716


In [7]:
moves_df["move_time"] = times_list
moves_df.head()

,game_id,move_no,move_no_pair,player,notation,move,from_square,to_square,piece,color,fen,is_check,is_check_mate,is_fifty_moves,is_fivefold_repetition,is_game_over,is_insufficient_material,white_count,black_count,white_pawn_count,black_pawn_count,white_queen_count,black_queen_count,white_bishop_count,black_bishop_count,white_knight_count,black_knight_count,white_rook_count,black_rook_count,captured_score_for_white,captured_score_for_black,fen_row1_white_count,fen_row2_white_count,fen_row3_white_count,fen_row4_white_count,fen_row5_white_count,fen_row6_white_count,fen_row7_white_count,fen_row8_white_count,fen_row1_white_value,fen_row2_white_value,fen_row3_white_value,fen_row4_white_value,fen_row5_white_value,fen_row6_white_value,fen_row7_white_value,fen_row8_white_value,fen_row1_black_count,fen_row2_black_count,fen_row3_black_count,fen_row4_black_count,fen_row5_black_count,fen_row6_black_count,fen_row7_black_count,fen_row8_black_count,fen_row1_black_value,fen_row2_black_value,fen_row3_black_value,fen_row4_black_value,fen_row5_black_value,fen_row6_black_value,fen_row7_black_value,fen_row8_black_value,move_sequence,move_time
0,ace8ccd9-9e96-45d3-b048-258be0c3fb01,1,1,Sarabtl,e4,e2e4,e2,e4,P,White,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR,0,0,0,0,0,0,16,16,8,8,1,1,2,2,2,2,2,2,0,0,8,7,0,1,0,0,0,0,31,7,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0,0,0,0,0,0,8,31,e4,0.0
1,ace8ccd9-9e96-45d3-b048-258be0c3fb01,2,1,KINGJarthur,e6,e7e6,e7,e6,P,Black,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBNR,0,0,0,0,0,0,16,16,8,8,1,1,2,2,2,2,2,2,0,0,8,7,0,1,0,0,0,0,31,7,0,1,0,0,0,0,0,0,0,0,0,1,7,8,0,0,0,0,0,1,7,31,e4|e6,0.0
2,ace8ccd9-9e96-45d3-b048-258be0c3fb01,3,2,Sarabtl,Nc3,b1c3,b1,c3,N,White,rnbqkbnr/pppp1ppp/4p3/8/4P3/2N5/PPPP1PPP/R1BQKBNR,0,0,0,0,0,0,16,16,8,8,1,1,2,2,2,2,2,2,0,0,7,7,1,1,0,0,0,0,28,7,3,1,0,0,0,0,0,0,0,0,0,1,7,8,0,0,0,0,0,1,7,31,e4|e6|Nc3,2.271
3,ace8ccd9-9e96-45d3-b048-258be0c3fb01,4,2,KINGJarthur,b6,b7b6,b7,b6,P,Black,rnbqkbnr/p1pp1ppp/1p2p3/8/4P3/2N5/PPPP1PPP/R1B...,0,0,0,0,0,0,16,16,8,8,1,1,2,2,2,2,2,2,0,0,7,7,1,1,0,0,0,0,28,7,3,1,0,0,0,0,0,0,0,0,0,2,6,8,0,0,0,0,0,2,6,31,e4|e6|Nc3|b6,2.164
4,ace8ccd9-9e96-45d3-b048-258be0c3fb01,5,3,Sarabtl,Qf3,d1f3,d1,f3,Q,White,rnbqkbnr/p1pp1ppp/1p2p3/8/4P3/2N2Q2/PPPP1PPP/R...,0,0,0,0,0,0,16,16,8,8,1,1,2,2,2,2,2,2,0,0,6,7,2,1,0,0,0,0,19,7,12,1,0,0,0,0,0,0,0,0,0,2,6,8,0,0,0,0,0,2,6,31,e4|e6|Nc3|b6|Qf3,2.466


## Add Stockfish centipawn score for each move. A good move makes the score more positive for White, and more negative for Black.

In [39]:
prev_id = str(moves_df.loc[0, ["game_id"]])
move_scores = []
count = 0

board = chess.Board("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1")

for i in range(len(moves_df)):
    move_info = moves_df.loc[i, ["game_id", "move_no", "move"]]
    if move_info[0] != prev_id: # check if there is a new game, reset prev's & board position
        prev_id = move_info[0]
        board = chess.Board("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1")
        count += 1
        if count % 100 == 0:
            print("Processed games: " + str(count))
    
    board.push_san(move_info[2])
    move_scores.append(str(engine.analyse(board, chess.engine.Limit(depth=5)).get("score").white()))
print(str(move_scores))

Processed games: 100
Processed games: 200
Processed games: 300
Processed games: 400
Processed games: 500
Processed games: 600
Processed games: 700
Processed games: 800
Processed games: 900
Processed games: 1000
Processed games: 1100
Processed games: 1200
Processed games: 1300
Processed games: 1400
Processed games: 1500
Processed games: 1600
Processed games: 1700
Processed games: 1800
Processed games: 1900
Processed games: 2000
Processed games: 2100
Processed games: 2200
Processed games: 2300
Processed games: 2400
Processed games: 2500
Processed games: 2600
Processed games: 2700
Processed games: 2800
Processed games: 2900
Processed games: 3000
Processed games: 3100
Processed games: 3200
Processed games: 3300
Processed games: 3400
Processed games: 3500
Processed games: 3600
Processed games: 3700
Processed games: 3800
Processed games: 3900
Processed games: 4000
Processed games: 4100
Processed games: 4200
Processed games: 4300
Processed games: 4400
Processed games: 4500
Processed games: 46

Processed games: 36200
Processed games: 36300
Processed games: 36400
Processed games: 36500
Processed games: 36600
Processed games: 36700
Processed games: 36800
Processed games: 36900
Processed games: 37000
Processed games: 37100
Processed games: 37200
Processed games: 37300
Processed games: 37400
Processed games: 37500
Processed games: 37600
Processed games: 37700
Processed games: 37800
Processed games: 37900
Processed games: 38000
Processed games: 38100
Processed games: 38200
Processed games: 38300
Processed games: 38400
Processed games: 38500
Processed games: 38600
Processed games: 38700
Processed games: 38800
Processed games: 38900
Processed games: 39000
Processed games: 39100
Processed games: 39200
Processed games: 39300
Processed games: 39400
Processed games: 39500
Processed games: 39600
Processed games: 39700
Processed games: 39800
Processed games: 39900
Processed games: 40000
Processed games: 40100
Processed games: 40200
Processed games: 40300
Processed games: 40400
Processed g

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [40]:
moves_df["centipawn_score"] = move_scores
moves_df.head()

,game_id,move_no,move_no_pair,player,notation,move,from_square,to_square,piece,color,fen,is_check,is_check_mate,is_fifty_moves,is_fivefold_repetition,is_game_over,is_insufficient_material,white_count,black_count,white_pawn_count,black_pawn_count,white_queen_count,black_queen_count,white_bishop_count,black_bishop_count,white_knight_count,black_knight_count,white_rook_count,black_rook_count,captured_score_for_white,captured_score_for_black,fen_row1_white_count,fen_row2_white_count,fen_row3_white_count,fen_row4_white_count,fen_row5_white_count,fen_row6_white_count,fen_row7_white_count,fen_row8_white_count,fen_row1_white_value,fen_row2_white_value,fen_row3_white_value,fen_row4_white_value,fen_row5_white_value,fen_row6_white_value,fen_row7_white_value,fen_row8_white_value,fen_row1_black_count,fen_row2_black_count,fen_row3_black_count,fen_row4_black_count,fen_row5_black_count,fen_row6_black_count,fen_row7_black_count,fen_row8_black_count,fen_row1_black_value,fen_row2_black_value,fen_row3_black_value,fen_row4_black_value,fen_row5_black_value,fen_row6_black_value,fen_row7_black_value,fen_row8_black_value,move_sequence,move_time,centipawn_score
0,ace8ccd9-9e96-45d3-b048-258be0c3fb01,1,1,Sarabtl,e4,e2e4,e2,e4,P,White,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR,0,0,0,0,0,0,16,16,8,8,1,1,2,2,2,2,2,2,0,0,8,7,0,1,0,0,0,0,31,7,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0,0,0,0,0,0,8,31,e4,0.0,+44
1,ace8ccd9-9e96-45d3-b048-258be0c3fb01,2,1,KINGJarthur,e6,e7e6,e7,e6,P,Black,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBNR,0,0,0,0,0,0,16,16,8,8,1,1,2,2,2,2,2,2,0,0,8,7,0,1,0,0,0,0,31,7,0,1,0,0,0,0,0,0,0,0,0,1,7,8,0,0,0,0,0,1,7,31,e4|e6,0.0,+85
2,ace8ccd9-9e96-45d3-b048-258be0c3fb01,3,2,Sarabtl,Nc3,b1c3,b1,c3,N,White,rnbqkbnr/pppp1ppp/4p3/8/4P3/2N5/PPPP1PPP/R1BQKBNR,0,0,0,0,0,0,16,16,8,8,1,1,2,2,2,2,2,2,0,0,7,7,1,1,0,0,0,0,28,7,3,1,0,0,0,0,0,0,0,0,0,1,7,8,0,0,0,0,0,1,7,31,e4|e6|Nc3,2.271,+25
3,ace8ccd9-9e96-45d3-b048-258be0c3fb01,4,2,KINGJarthur,b6,b7b6,b7,b6,P,Black,rnbqkbnr/p1pp1ppp/1p2p3/8/4P3/2N5/PPPP1PPP/R1B...,0,0,0,0,0,0,16,16,8,8,1,1,2,2,2,2,2,2,0,0,7,7,1,1,0,0,0,0,28,7,3,1,0,0,0,0,0,0,0,0,0,2,6,8,0,0,0,0,0,2,6,31,e4|e6|Nc3|b6,2.164,+97
4,ace8ccd9-9e96-45d3-b048-258be0c3fb01,5,3,Sarabtl,Qf3,d1f3,d1,f3,Q,White,rnbqkbnr/p1pp1ppp/1p2p3/8/4P3/2N2Q2/PPPP1PPP/R...,0,0,0,0,0,0,16,16,8,8,1,1,2,2,2,2,2,2,0,0,6,7,2,1,0,0,0,0,19,7,12,1,0,0,0,0,0,0,0,0,0,2,6,8,0,0,0,0,0,2,6,31,e4|e6|Nc3|b6|Qf3,2.466,+1


In [42]:
moves_cols = ["game_id", "player", "move_no", 
              "move", "piece", "fen", 
              "is_check", "is_check_mate", "is_fifty_moves", 
              "is_game_over", "captured_score_for_white", "captured_score_for_black",
              "move_time", "centipawn_score"]
moves_df = moves_df[moves_cols]
moves_df.head()

,game_id,player,move_no,move,piece,fen,is_check,is_check_mate,is_fifty_moves,is_game_over,captured_score_for_white,captured_score_for_black,move_time,centipawn_score
0,ace8ccd9-9e96-45d3-b048-258be0c3fb01,Sarabtl,1,e2e4,P,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR,0,0,0,0,0,0,0.0,+44
1,ace8ccd9-9e96-45d3-b048-258be0c3fb01,KINGJarthur,2,e7e6,P,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBNR,0,0,0,0,0,0,0.0,+85
2,ace8ccd9-9e96-45d3-b048-258be0c3fb01,Sarabtl,3,b1c3,N,rnbqkbnr/pppp1ppp/4p3/8/4P3/2N5/PPPP1PPP/R1BQKBNR,0,0,0,0,0,0,2.271,+25
3,ace8ccd9-9e96-45d3-b048-258be0c3fb01,KINGJarthur,4,b7b6,P,rnbqkbnr/p1pp1ppp/1p2p3/8/4P3/2N5/PPPP1PPP/R1B...,0,0,0,0,0,0,2.164,+97
4,ace8ccd9-9e96-45d3-b048-258be0c3fb01,Sarabtl,5,d1f3,Q,rnbqkbnr/p1pp1ppp/1p2p3/8/4P3/2N2Q2/PPPP1PPP/R...,0,0,0,0,0,0,2.466,+1


In [43]:
moves_df.to_csv("2020January_moves.csv")